In [3]:
import numpy as np
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)
import matplotlib.pyplot as plt

array([20., 20., 20., 20., 20., 20., 20., 20., 20., 20.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 20., 20., 20.,
       20., 20., 20., 20., 20., 20., 20.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.])

In [22]:
# Pontos notáveis
A = np.array([ 0,  0])
B = np.array([ 5,  0])
C = np.array([ 9,  0])
D = np.array([18,  0])
E = np.array([36,  0])
F = np.array([36, 18])
G = np.array([36, -5]) # ponto criado artificialmente
H = np.array([45,  0]) # ponto criado artificialmente
I = np.array([65,  0]) # ponto criado artificialmente

# Dimensões vigas/treliças
A1 = ( 1.8, 0.9) # torre
A2 = ( 0.9, 0.9) # ponte
D3 = (0.05,)     # treliças

# Valores iniciais de N1 e N2
N1 = 20
N2 = 20

# Lambda functions para o valor da carga distribuida entre A-C e entre E-H
# L é parâmetro desta fórmula para o caso em que queremos dividir os elementos com
# carga concentrada em sub-elementos, os quais terão suas próprias cargas distribuidas
q = lambda N, L = N*80*9.8/L

# piece-wise function para determinar o valor de N em cada parte da plataforma
x = np.linspace(0)


def angle(u, v=np.array([1, 0])):
    '''
    Função que calcula o ângulo entre dois vetores
    
    Parameters
    ----------
    u: np.ndarray, dim=(2, )
    v: np.ndarray (default=np.array([1, 0])), dim=(2, )
        vetor horizontal para direita
    
    Returns
    -------
    angle: float
        valor, em radianos, do ângulo entre os vetores
    '''
    u_norm = u/np.linalg.norm(u)
    v_norm = v/np.linalg.norm(v)
    dot    = np.dot(u_norm, v_norm)
    angle  = np.arccos(dot)
    
    return angle

def calculate_I(b, h=None):
    '''
    Função que calcula o momento de inércia
    
    Parameters
    ----------
    b: float
        largura da seção transversal (ou diâmetro)
    h: float (default=None)
        altura da seção transversal, se None considera que trata-se
        de uma seção transversal circular
    '''
    if h is None:
        return np.pi*b**4/12
    return b*h**3/12

def globalize_K(Ke, total_size, idx):
    '''
    Função que transforma uma matriz Ke (local) em uma matriz K (global)
    
    Parameters
    ----------
    Ke: np.array (dim=len(idx)xlen(idx))
        Matriz de rigidez local
    total_size: int
        tamanho total do sistema (número de variáveis independentes)
    idx: list (or array-like)
        índice das variáveis que Ke se refere a
    '''
    Keg = np.zeros((total_size, total_size), dtype='complex')
    Keg[np.ix_(idx, idx)] = Ke
    
    return Keg

def anula_cond_contorno(A, idx):
    '''
    Anula as linhas e colunas em que se sabe que o movimento/rotação
    é nulo, deixando 1 no lugar
    
    Parameters
    ----------
    A: np.ndarray
        matriz global
    idx: list
        lista dos índices (inteiros, 0-indexados)
    '''
    A_cc = A.copy()
    for var in idx:
        A_cc[var,   :] = 0
        A_cc[:  , var] = 0
        A_cc[var, var] = 1
    return A_cc

def reduz_matriz(A, idx):
    '''
    Reduz a matriz global com condições de contorno para
    uma matriz apenas com as linhas e colunas dos elementos
    não triviais (não nulos) do sistema
    
    Parameters
    ----------
    A: np.ndarray
        matriz global
    idx: list
        lista dos índices (inteiros, 0-indexados)
    '''
    A_red = A.copy()
    A_red = A[np.ix_(idx, idx)]
    
    return A_red


# Funções de Hermite
H1   = lambda x, L: 1 - 3*x**2/L**2 + 2*x**3/L**3 
dH1  = lambda x, L: -6*x/L**2 + 6*x**2/L**3
d2H1 = lambda x, L: -6/L**2 + 12*x/L**3
H2   = lambda x, L: x - 2*x**2/L + x**3/L**2
dH2  = lambda x, L: 1 - 4*x/L + 3*x**2/L**2
d2H2 = lambda x, L: -4/L + 6*x/L**2
H3   = lambda x, L: 3*x**2/L**2 - 2*x**3/L**3
dH3  = lambda x, L: 6*x/L**2 - 6*x**2/L**3
d2H3 = lambda x, L: 6/L**2 - 12*x/L**3
H4   = lambda x, L: -x**2/L + x**3/L**2
dH4  = lambda x, L: -2*x/L + 3*x**2/L**2
d2H4 = lambda x, L: -2/L + 6*x/L**2

In [ ]:
class Elemento:
    '''
    Classe para um Elemento, podendo ele ser de viga ou de treliça
    '''
    def __init__(ini, fim, tipo, params, q):
        '''
        Parameters
        ----------
        ini: np.ndarray
            coordenada do início do elemento
        fim: np.ndarray
            coordenada do fim do elemento
        tipo: str {'viga', 'trelica'}
            tipo de elemento a ser estudado
        params: dict
            dicionário com os seguintes parâmetros (no SI):
            - A: área transversal do elemento
            - E: módulo de Young do elemento
            - rho: densidade do elemento
            - I: momento de inércia do elemento
            outros parâmetros relevantes (como tamanho L e o ângulo de incli-
            nação theta são determinados através de contas com os parâmetros 
            já dados)
        q: float
            valor da carga distribuída no elemento
        '''
        self.ini    = ini
        self.fim    = fim
        self.tipo   = tipo
        self.L      = self.calculate_L()
        self.theta  = self.calculate_theta()
        (self.E  ,
         self.A  ,
         self.rho,
         self.I  ,) = params.values()
        self.q      = q
        self.Ke     = self.build_Ke_matrix()
        self.Me     = self.build_Me_matrix()
        self.Fe     = self.build_q_array()
    
    def calculate_L(self, ):
        '''
        Método que calcula o tamanho L do elemento
        '''
        return np.linalg.norm(self.ini - self.fim)
        
    def calculate_theta(self, ):
        '''
        Método que calcula a inclinação do elemento através das
        coordenadas de seu início e seu fim
        '''
        u = self.fim - self.ini
        return angle(u)
    
    def rotate_matrix(self,):
        '''
        Método que cria uma matriz de rotação para o elemento criado
        a partir do ângulo theta
        
        Notes
        -----
        A matriz rotacionada é obtida a partir da seguinte expressao:
        .. math:: [M]_{rot} = [T]^{T}\cdot [M] \cdot [T]
        Com [M] a matriz original (em seu próprio eixo coordenado) e
        [M]_{rot} a matriz rotacionada, sendo [M] uma matriz quadrada
        No caso que querermos rotacionar uma matriz-coluna (carga), 
        deve-se aplicar a seguinte equação:
        .. math:: [C]_{rot} = [T]^{T}\cdot [C]
        '''
        c = np.cos(self.theta)
        s = np.sin(self.theta)
        T = np.array([
            [ c, s, 0,  0, 0, 0], # u1
            [-s, c, 0,  0, 0, 0], # v1
            [ 0, 0, 1,  0, 0, 0], # phi1
            [ 0, 0, 0,  c, s, 0], # u2
            [ 0, 0, 0, -s, c, 0], # v2
            [ 0, 0, 0,  0, 0, 1], # phi2
        ])
        return T
        
    def build_Ke_matrix(self,):
        '''
        Método que constrói a matriz de elasticidade local do elemento
        '''
        E = self.E
        A = self.A
        L = self.L
        I = self.I
        if self.tipo == 'trelica':
            Ke = E*A/L*np.array([
                [ 1, 0, 0, -1, 0, 0], # u1
                [ 0, 0, 0,  0, 0, 0], # v1
                [ 0, 0, 0,  0, 0, 0], # phi1
                [-1, 0, 0,  1, 0, 0], # u2
                [ 0, 0, 0,  0, 0, 0], # v2
                [ 0, 0, 0,  0, 0, 0], # phi2
            ])
        else:
            Ke = E*I/L**3*np.array([
                [0,   0,      0, 0,    0,      0], # u1
                [0,  12,    6*L, 0,  -12,    6*L], # v1
                [0, 6*L, 4*L**2, 0, -6*L, 2*L**2], # phi1
                [0,   0,      0, 0,    0,      0], # u2
                [0, -12,   -6*L, 0,   12,   -6*L], # v2
                [0, 6*L, 2*L**2, 0, -6*L, 4*L**2], # phi2
            ])
        T   = self.rotate_matrix()
        Ke_ = T.T @ Ke @ T
            
        return Ke_
    
    def build_Me_matrix(self,):
        '''
        Método que constrói a matriz de massas local do elemento
        '''
        E = self.E
        A = self.A
        L = self.L
        I = self.I
        rho = self.rho
        if self.tipo == 'trelica':
            Me = rho*A*L/6*np.array([
                [2, 0, 0, 1, 0, 0], # u1
                [0, 0, 0, 0, 0, 0], # v1
                [0, 0, 0, 0, 0, 0], # phi1
                [1, 0, 0, 2, 0, 0], # u2
                [0, 0, 0, 0, 0, 0]  # v2
                [0, 0, 0, 0, 0, 0], # phi2
            ])
        else:
            Me = rho*A*L/420*np.array([
                [0,    0,       0, 0,     0,       0], # u1
                [0,  156,    22*L, 0,    54,   -13*L], # v1
                [0, 22*L,  4*L**2, 0,  13*L, -3*L**2], # phi1
                [0,    0,       0, 0,     0,       0], # u2
                [0,   54,    13*L, 0,   156,   -22*L], # v2
                [0,-13*L, -3*L**2, 0, -22*L,  4*L**2], # phi2
            ])
        T   = self.rotate_matrix()
        Me_ = T.T @ Me @ T
        
        return Me_
    
    def build_q_array(self,):
        '''
        Método que constrói a matriz-coluna de forças distribuídas
        local do elemento
        '''
        q  = self.q
        L  = self.L
        Fe = q*L/12*np.array([6, L, 6, -L])
        
        T   = self.rotate_matrix()
        Fe_ = T.T @ Fe
        
        return Fe_

In [35]:
x = np.linspace(0, 65, 66)
cargas = np.logical_or(x <= 9, np.logical_and(x >= 36, x<=45))
N = np.piecewise(x, cargas, [20, 0])

t = [np.array([0, 0]), np.array([3, 4])]
theta = angle(t[1] - t[0])
l = np.linalg.norm(t[1] - t[0])
ant = t[0]
while l > dx:
    atual = ant + np.array([np.cos(theta), np.sin(theta)])
    q_ = q(N[atual[0]], dx)
    e = Elemento(ant, atual, 'trelica', {'E': 1, 'A': 1, 'rho': 1, 'I': 1}, q)
    
#dx = 3

1.3333333333333335

In [26]:
tamanho = np.hypot(t[0], t[1])
tamanho

array([5., 0.])

In [33]:
np.linalg.norm(t[1] - t[0])

4.0

In [ ]:
# (a.i)
dx_list = [1, 2, 3]
dx = 3
# Macro-elementos (a serem discretizados, ou não)
# Plataforma (viga)
# A-B-C-D-E-I
# a serem discretizados por partes
# Torre (viga)
# G-E-F
# a serem discretizados por partes
# Cabos (treliça)
# 1: B-F
# 2: D-F
# 3: I-F
# que não devem ser discretizados!

# lista com os elementos de pórtico
elementos = []